## 🛠️ **ETL (Extract, Transform, Load)**
### **📂Procesamos el 3er archivo: `user_reviews.json.gz`**


- 🗑️ Eliminado `reviews_funny` , `reviews_last_edited `
- `reviews_posted:`
  - Extraemos de las fechas el año en formato YYYY
  - Cambiamos de string a int
- 🗃️ `reviews`: Desanidado: era una lista de diccionarios

#### **Importamos las librerías que vamos a usar**


In [1]:
import pandas as pd  # Pandas se utiliza para el manejo y análisis de datos tabulares
import pyarrow as pa  # PyArrow se utiliza para trabajar con formatos de datos columnares y eficientes como Parquet
import pyarrow.parquet as pq  # Importamos Parquet
import nltk
nltk.download('vader_lexicon')

import ast  # AST (Abstract Syntax Trees) se utiliza para interpretar expresiones Python
import gzip
import json  # JSON se utiliza para trabajar con datos en formato JSON
import os  # OS proporciona funciones para interactuar con el sistema operativo
import time
import warnings  # Warnings se utiliza para gestionar advertencias y filtrarlas si es necesario
warnings.filterwarnings("ignore")
from data_utils import data_type_check, duplicados_columna
# Autoreload se utiliza para recargar automáticamente los módulos al realizar cambios
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package vader_lexicon to C:\Users\El
[nltk_data]     Bauto\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


#### **Descomprimimos todos los archivos gz** 

In [2]:
# Obtenemos el tiempo de inicio de todo este ipynb 
start_time = time.time()

def descomprimir_archivos_gz(archivos_gz, carpeta_destino):
    for archivo_gz in archivos_gz:
        with gzip.open(archivo_gz, 'rb') as f_in:
            contenido = f_in.read()
            archivo_destino = os.path.join(carpeta_destino, os.path.splitext(os.path.basename(archivo_gz))[0])
            with open(archivo_destino, 'wb') as f_out:
                f_out.write(contenido)
        print(f'Archivo descomprimido: {archivo_destino}')

# Ejemplo de uso con una lista de archivos gz
archivo_gz_a_descomprimir = ['../0 Dataset/user_reviews.json.gz']
carpeta_destino = '../0 Dataset/'

descomprimir_archivos_gz(archivo_gz_a_descomprimir, carpeta_destino)


Archivo descomprimido: ../0 Dataset/user_reviews.json


#### 📦 **EXTRACT**
- Tomamos los datos del archivo JSON, transformamos en un DataFrame y realizamos una primera observación de su contenido.

In [3]:
#Creamos una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("../0 Dataset/user_reviews.json","r", encoding='utf-8') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))

#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_user_reviews = pd.DataFrame(rows)
#Veamos unos registros al asar
df_user_reviews.sample(2)

,user_id,user_url,reviews
19390,moowaffles,http://steamcommunity.com/id/moowaffles,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
18019,webo2456,http://steamcommunity.com/id/webo2456,"[{'funny': '5 people found this review funny',..."


In [4]:
# Usando una funcion personalizada vemos las variables categóricas, numéricas, dimensiones del dataframe, nulos, tipos de datos   
df_info = data_type_check(df_user_reviews)


 Resumen del dataframe:

Dimensiones:  (25799, 3)
    columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0   user_id       100.0      0.0            0    object
1  user_url       100.0      0.0            0    object
2   reviews       100.0      0.0            0    object


### 🔁 **TRANSFORM**

#### user_id

Verificamos los duplicados en user_id

In [5]:
duplicados = duplicados_columna(df_user_reviews, 'user_id')
duplicados

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


623 filas duplicadas en user_id.

Es necesario verificar si la información de los comentarios anidados en la columna 'review' también se duplica, o si solo se duplica el 'user_id' porque un mismo usuario realizó varios comentarios.

In [6]:
#Eliminamos duplicados en reseñas, dejando la primer reseña
df_user_reviews = df_user_reviews.drop_duplicates(subset='user_id', keep='first')

# Se revisa un usuario de ejemplo
user_id = 'yolofaceguy'
user_reviews = duplicados[duplicados['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('-' * 40)

from the creaters of the walking dead, i present to you, the wolf among us. a twisted an unhappy place where crimes are made in the town of the fables. Ecperience one of the most mind-bending, jaw-dropping twists you will see in all of gaming history...well, some of gaming history. SPOILERS: i really liked how bigby turns into his true form to fight all the bloody marys... its just like when neo fought all the agent smiths in the matrix. what i didnt really like is when i chose to lock the crooked man up it glitches my game and i start from the near begining where bigby fought the crooked man's crew, exept the guy who runs a strip club wasnt there. so i was really confused. but anyway i really recommend this game if you want twists and ♥♥♥♥ed up scenes.
this game is awesome,this game is ♥♥♥♥ed up and this game is so sad and depressing. if you want these types of games i would strongly reccomend youto buy it. its worth your money
----------------------------------------
from the creater

Se observa que los comentarios son iguales, por lo que se eliminan los duplicados, manteniendo el primer registro

In [7]:
df_user_reviews = df_user_reviews.drop_duplicates(subset='user_id', keep='first')
duplicados_columna(df_user_reviews, 'user_id')

'No hay duplicados'

#### **Reviews**: 
Se encuentra anidada, es una lista de diccionarios. 
Generamos una columna por cada diccionario para posteriormente hacer un registro por cada diccionario.


In [8]:
#1_Se transforma a columnas cada elemento de las listas:

df_reviews2 = pd.json_normalize(df_user_reviews['reviews'])
# Se agrega el 'user_id' y 'user_url' a las columnas separadas 
df_reviews2 = pd.concat([df_user_reviews[['user_id', 'user_url']], df_reviews2], axis=1)


#2_Generamos un registro por cada diccionario, manteniendo en cada caso el usuario que lo genera

# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
df_reviews2 = pd.melt(df_reviews2, id_vars=['user_id', 'user_url'], 
    value_vars=list(range(9)),
    value_name='reviews')

#3_Para evitar registros None luego de hacer pd.melt: 

# Se eliminan las filas con valor None
df_reviews2 = df_reviews2.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
df_reviews2[df_reviews2['user_id']=='js41637']

#4_ Convertimos cada diccionario en columna: 

# Se separan por columnas cada una de las claves de reviews
df_user_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_user_reviews = df_user_reviews.add_prefix('reviews_')

#5_ Para no perder user_id y user_url los concatenamos al df
df_user_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_user_reviews], axis=1)
df_user_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Vemos que hay valores faltantes y revisamos si son nulos o son espacios


In [9]:
df_user_reviews['reviews_last_edited'][0]

''

Reemplazamos esos espacios por valores nulos

In [10]:
df_user_reviews.replace('', None, inplace=True)
data_type_check(df_user_reviews) 
print(" ")
print("Vemos que todavia hay faltantes:")
print("     reviews_funny:                   %86.24")
print("     reviews_last_edited              %89.72")
print("Procedemos a eliminarlos:")


 Resumen del dataframe:

Dimensiones:  (57397, 9)
               columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0              user_id      100.00     0.00            0    object
1             user_url      100.00     0.00            0    object
2        reviews_funny       13.76    86.24        49498    object
3       reviews_posted      100.00     0.00            0    object
4  reviews_last_edited       10.28    89.72        51499    object
5      reviews_item_id      100.00     0.00            0    object
6      reviews_helpful      100.00     0.00            0    object
7    reviews_recommend      100.00     0.00            0      bool
8       reviews_review       99.95     0.05           30    object
 
Vemos que todavia hay faltantes:
     reviews_funny:                   %86.24
     reviews_last_edited              %89.72
Procedemos a eliminarlos:


Eliminamos los faltantes en reviews_funny y reviews_last_edited y cambiamos el tipo de dato.

In [11]:
df_user_reviews = df_user_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])

print("Observamos las columnas que nos quedaron: ")
print(" ")
data_type_check(df_user_reviews) 

Observamos las columnas que nos quedaron: 
 

 Resumen del dataframe:

Dimensiones:  (57397, 7)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0            user_id      100.00     0.00            0    object
1           user_url      100.00     0.00            0    object
2     reviews_posted      100.00     0.00            0    object
3    reviews_item_id      100.00     0.00            0    object
4    reviews_helpful      100.00     0.00            0    object
5  reviews_recommend      100.00     0.00            0      bool
6     reviews_review       99.95     0.05           30    object


### reviews_item_id

In [12]:
#Cambiamos el nombre reviews_item_id por reviews_item_id
df_user_reviews['reviews_item_id'] = df_user_reviews['reviews_item_id'].astype('int64')

#### 📤 **LOAD**

In [13]:
#Guardamos los cambios en parquet
table = pa.Table.from_pandas(df_user_reviews)
ruta_parquet = os.path.join('..', '0 Dataset', '1.3_user_review_LISTO.parquet')
df_user_reviews.to_parquet(ruta_parquet)
print(f'Se guardó el archivo {ruta_parquet}')

Se guardó el archivo ..\0 Dataset\1.3_user_review_LISTO.parquet


In [14]:
#Dejamos informacion de muestra acerca de ese archivo
data_type_check(df_user_reviews)


 Resumen del dataframe:

Dimensiones:  (57397, 7)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0            user_id      100.00     0.00            0    object
1           user_url      100.00     0.00            0    object
2     reviews_posted      100.00     0.00            0    object
3    reviews_item_id      100.00     0.00            0     int64
4    reviews_helpful      100.00     0.00            0    object
5  reviews_recommend      100.00     0.00            0      bool
6     reviews_review       99.95     0.05           30    object


## **Terminamos**
  - Eliminamos el archivo descomprimidos que ahora tenemos limpio y liviano en formato parquet

In [15]:
# Lista de archivos descomprimidos
gz_descomprimido = [
    '../0 Dataset/user_reviews.json'
    ]

# Eliminar archivos descomprimidos
for archivo_json in gz_descomprimido:
    try:
        os.remove(archivo_json)
        print(f'Archivo eliminado: {archivo_json}')
    except FileNotFoundError:
        print(f'Archivo no encontrado: {archivo_json}')

Archivo eliminado: ../0 Dataset/user_reviews.json


Observamos el tiempo de ejecucion total de nuestro proceso ETL 🔥

In [16]:
# Obtener el tiempo de finalización
end_time = time.time()
# Calcular el tiempo total de ejecución
total_time = end_time - start_time
# Convertir a minutos y redondear a 2 decimales
total_time_minutes = round(total_time / 60, 2)
# Imprimir resultados
print(f"Tiempo total de ejecución de este ipynb: {total_time_minutes} minutos")

Tiempo total de ejecución de este ipynb: 0.52 minutos
